In [1]:
import tensorflow as tf
import utils
import os
import glob

from dataset import get_datasets
from stormer import Stormer

2024-04-22 16:08:48.817724: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 16:08:48.859929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 16:08:49.554570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/zuherj/miniconda3/envs/kws/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_

In [2]:
# print model names

print_model_table = lambda model_list: utils.print_enumerated_list(model_list, "Model")

models_names = [path.split("/")[-1] for path in glob.glob("models/stormer*")]
models_names.sort()
print_model_table(models_names)
model_name = models_names[int(input("Choose model: "))]

Available Models:
-------------------------------------------------
| Index        | Model Name                     |
|--------------|--------------------------------|
| 0            | stormer_r2_h4_dm32_dataset=mel |
| 1            | stormer_r3_h4_dm32_dataset=mel |
|______________|________________________________|


ValueError: invalid literal for int() with base 10: ''

In [ ]:
hps = utils.load_hps(model_name)
stormer = Stormer(**hps)

2024-04-22 15:20:58.674797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 15:20:58.691083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 15:20:58.691225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
# check if the model containing directory exists
model_path = utils.get_model_path(model_name)
load_weights = os.path.exists(os.path.dirname(model_path))
if load_weights:
    stormer.load_weights(model_path)

In [ ]:
## load the datasets
train, valid, test = get_datasets(
    **hps
)

Loading dataset version 2
Dataset loaded


2024-04-22 15:20:59.939952: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 FMA


In [ ]:
# for layer in stormer.layers[:-1]:
#     print(layer.name)
#     layer.trainable = False

dense
rotary_positional_encoding
stormer_ru
stormer_ru_1


In [ ]:
results_filename = f'data/results/{model_name}.csv'

metrics=["accuracy"]

stormer.compile(
    optimizer=tf.keras.optimizers.AdamW(hps["learning_rate"]),
    loss="categorical_crossentropy",
    metrics=metrics,
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

state_transformer_history = stormer.fit(
    train,
    validation_data=valid,
    epochs=hps["num_epochs"],
    callbacks=[
        model_checkpoint_callback,
        utils.MetricsLogger(
            results_filename,
        )
    ],
)

Epoch 1/10000


2024-04-22 15:21:16.862444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-04-22 15:21:16.902172: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5579600929a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-22 15:21:16.902196: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-04-22 15:21:16.906209: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-22 15:21:16.983447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-04-22 15:21:17.045980: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.

 637/1489 [===========>..................] - ETA: 59s - loss: 5.2254 - accuracy: 0.0293

In [ ]:
stormer.summary()

Model: "stormer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  1312      
                                                                 
 rotary_positional_encoding  multiple                  0         
  (RotaryPositionalEncoding                                      
 )                                                               
                                                                 
 stormer_ru (StormerRU)      multiple                  51008     
                                                                 
 stormer_ru_1 (StormerRU)    multiple                  51008     
                                                                 
 dense_61 (Dense)            multiple                  1188      
                                                                 
Total params: 104516 (408.27 KB)
Trainable params: 1188 (4.